In [2]:
import torch
import numpy as np
import os,sys
if "../.." not in sys.path:
    sys.path.append("../..")
from transformers import AutoModelForCausalLM, AutoConfig

from model_api import CustomModelHandler, format_prompt
from model import CustomLLaMA, CustomLlamaConfig
from test_on_struq import *

# import from 2 directories up
notebook_dir = os.getcwd()
# Go two directories up
parent_dir = os.path.abspath(os.path.join(notebook_dir, "../.."))
sys.path.append(parent_dir)
from eval import *

In [11]:
data_path = "data/tatsu-lab/alpaca_farm/eval.json"
with open(data_path, "r") as f:
    data = json.load(f)
with open("data/prompt_templates.json", "r") as f:
    templates = json.load(f)
template = templates[0]

In [3]:
embedding_type = 'forward_rot'
base_model =  "Qwen/Qwen2.5-7B"
model_path = "Qwen2.5-7B_ASIDE"
device = 'cuda'
handler = CustomModelHandler(
    model_path, 
    base_model, 
    base_model, 
    model_path, 
    None,
    0, 
    embedding_type=embedding_type, 
    load_from_checkpoint=True
)
handler.model.to(device)
attack = 'id_naive'

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.



 <class 'model.CustomQwenConfig'> <class 'model.QwenForwardRot'> 

CALLED load_vanilla_model_and_tokenizer on model Qwen2.5-7B_ASIDE and tokenizer Qwen2.5-7B_ASIDE
Model config CustomQwenConfig {
  "add_linear_shift": false,
  "architectures": [
    "QwenForwardRot"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "gradual_rotation": false,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "learned_rotation": false,
  "max_position_embeddings": 131072,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "pad_token_id": 151643,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "rotation_alpha": 1.57079633,
  "rotation_direction": "right",
  "sliding_window": 131072,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": fal

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

chat_template_path: None

 MODEL TYPE:  <class 'model.QwenForwardRot'>


In [ ]:
def generate_responses_batch(handler, template, examples, batch_size, injection_method):
    """
    Generate model responses for a batch of examples with injected attacks.
    
    This function processes evaluation examples in batches, applying the specified
    injection method to each example and collecting model responses for analysis.
    
    Args:
        handler (CustomModelHandler): Initialized model handler
        template (dict): Prompt template for formatting
        examples (list): List of evaluation examples with 'instruction' and 'input' keys
        batch_size (int): Number of examples to process per batch
        injection_method (function): Attack function to apply to each input
        
    Returns:
        list: Model responses for all examples
        
    Workflow:
        1. Apply injection method to each example's input data
        2. Format prompts using template system for proper instruction-data separation
        3. Generate responses using batch API for efficiency
        4. Collect all responses for success rate analysis
        
    Note:
        Debug information is printed for the first example to verify attack application.
    """
    responses = []
    for start_idx in tqdm(range(0, len(examples), batch_size)):
        end_idx = min(start_idx + batch_size, len(examples))
        batch_data = deepcopy(examples[start_idx:end_idx])


        inst_list = []
        data_list = []
        for i, example in enumerate(batch_data):

            clean_input = example["input"]
            injected_sample = injection_method(example)
            instruction = injected_sample["instruction"]
            injected_input = injected_sample["input"]

            # if start_idx == 0 and i == 0:
            #     # For debugging purposes, print the first example
            #     print(f"Injection method: {injection_method.__name__}")
            #     print(f"Instruction: {instruction}")
            #     print(f"Clean input: {clean_input}")
            #     print(f"Injected input: {injected_input}")

            instruction_text = format_prompt(instruction, template, "system")
            data_text = format_prompt(injected_input, template, "user")
            inst_list.append(instruction_text)
            data_list.append(data_text)

        batch_responses, batch_inps = handler.call_model_api_batch(
            inst_list, data_list,max_new_tokens=512,do_sample = False
        ) 

        responses.extend(batch_responses)

    return responses 

In [16]:
attack = 'naive'
responses = generate_responses_batch(handler,template,data[:8],8,eval(attack))


  0%|                                                                                                                                                                                                                                | 0/1 [00:02<?, ?it/s]

Generation time: 2.780323 seconds
